# Automated TMA scanning
Procedure for automated tissue microarray scanning on Leica SP8.

## Import libraries

In [ ]:
%run ../utils.ipynb

In [ ]:
# communication with microscope
from leicacam import CAM
cam = CAM()

In [ ]:
import os
from leicaexperiment import Experiment

In [ ]:
from leicaexperiment import attributes
from leicaautomator.utils import stitch
from skimage.io import imsave

In [6]:
from leicaautomator import find_tma_regions, construct_stage_position

In [ ]:
from leicaautomator.utils import zick_zack_sort, save_regions

## Adjusting specimen-plane
- Measure each corner of your sample and adjust the specimen-plane accordingly

In [ ]:
top_left = cam.get_information('zdrive')['zpos']

In [ ]:
bottom_left = cam.get_information('zdrive')['zpos']

In [ ]:
bottom_right = cam.get_information('zdrive')['zpos']

In [ ]:
top_right = cam.get_information('zdrive')['zpos']

In [ ]:
top_left, top_right, bottom_left, bottom_right

## Overview images
- define and start the overview scan before running blocks

In [ ]:
export_root = r'd:\MatrixScreenerImages' + '\\'

In [ ]:
relpath = cam.wait_for('relpath')['relpath']

In [ ]:
experiment_path = os.path.join(export_root, *relpath.split(os.sep)[:-4])

In [ ]:
experiment = Experiment(experiment_path)
cam.wait_for('inf', 'scanfinished')
experiment.compress(delete_tif=True);

### rotate

In [ ]:
for i in experiment.images:
    img = Image.open(i)
    img = img.rotate(270)
    img.save(i)

### equalize

In [ ]:
experiment = equalize_experiment(experiment) # code in utils.ipynb

### stitch

In [ ]:
stitched, offset = stitch(experiment)
stitched_filename = experiment.path+os.sep+'stitched--U'+attr.U+'V--'+attr.V+'C--'+attr.C+'Z--'+attr.Z+'.png'
imsave(stitched_filename, stitched)

## Find regions to scan

In [7]:
tma_regions = find_tma_regions(stitched)
stage_position = construct_stage_position(experiment, offset)
stage_position(1000, 2456) # stage position of pixel (1000, 2456) (y, x)

## Scan

In [ ]:
tmpl_path = r"C:\Users\TCS-User\AppData\Roaming\Leica Microsystems\LAS X\MatrixScreener\ScanningTemplates" + "\\"
tmpl_name = tmpl_path + '{ScanningTemplate}leicaautomator'

### calc start position and number of fields to scan

In [ ]:
tmpl = ScanningTemplate(tmpl_name + '0.xml')
y_distance = tmpl.properties.ScanFieldStageDistanceY * 1e-6 # in microns
x_distance = tmpl.properties.ScanFieldStageDistanceX * 1e-6

# check that alternating templates match
tmpl = ScanningTemplate(tmpl_name + '1.xml')
assert y_distance == tmpl.properties.ScanFieldStageDistanceY * 1e-6 # in microns
assert x_distance == tmpl.properties.ScanFieldStageDistanceX * 1e-6

for r in regions:
    minpos = stage_position(r.y, r.x)
    maxpos = stage_position(r.y_end, r.x_end)
    r.y_start = minpos[0] - y_distance/2
    r.x_start = minpos[1] - x_distance/2
    r.y_fields = (maxpos[0] - minpos[0]) / y_distance
    r.x_fields = (maxpos[1] - minpos[1]) / x_distance

In [ ]:
# regions sorted as [r(1,1), r(1,2), r(2,2), r(2,1), r(3,1), r(3,2), ...] # here r(2,1) is region(col=2, row=1)
regions = zick_zack_sort(regions, ('well_x', 'well_y'))

for n, region in enumerate(regions):
    # alternate between tmpl_name0/1.xml
    # LAS cannot load same filename twice
    tmpl = ScanningTemplate(tmpl_name + str(n%2) + '.xml')
    
    # set start position and number of fields
    tmpl.move_well(1, 1, region.x_start, region.y_start)
    tmpl.enable_fields((region.y_fields, region.x_fields))

    # autofocus
    tmpl.write()
    cam.load_template(tmpl.filename)
    cam.autofocus_scan()
    cam.wait_for('inf', 'scanfinished')

    # scan
    cam.start_scan()

    # record output filename
    relpath = cam.wait_for('relpath')['relpath']
    region.folder = os.path.join(export_root, *relpath.split(os.sep)[:-4])
    
    # wait until finished
    cam.wait_for('inf', 'scanfinished')
    
    print('Scanned (%s,%s) (x, y)' % (region.well_x, region.well_y))

In [ ]:
# keep experiment data
save_regions('regions.pickle.gz', regions)